<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/%E2%98%85CareerLength_Stathead_com_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 문제상황: 링크로 연결된 url로 가서 데이터를 가져 오고 있지만 다시 처음의 페이지로 돌아와 다음 행의 선수 링크 url로 이동하지 못하고 있음. (에러 첨부)
-> 여러가지 문제가 복합적으로 있어서 좀 많이 고쳤습니다.
-> 일단 매번 뒤로가기 하는건 속도측면에서 비효율적이라. url먼저 수집해두고 도는 방식으로 수정 했습니다.
-> 선수 페이지로 넘어가면 도메인이 변경되면서 로그인이 풀리는 이슈가 있어 로그인 로직을 추가 했습니다.
-> 페이지 넘기는 기능은 offset값만 바꾸는 방식으로 변경 했습니다.
-> 안타깝게도... colab에서 undetected_chromedriver를 정상동작 시키는 방법을 찾지 못해서 주피터에서만 동작하는 코드를 공유 드립니다.



In [ ]:
!pip install selenium
!pip install undetected_chromedriver
!pip install webdriver_manager
!pip install pandas

In [ ]:
#수정 후 코드
#쥬피터 OK
#draft list의 선수별 데이터 가져오기(career length)

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time
import os

def get_season_from_page(soup):
    season_element = soup.select_one("#meta div h1 span")
    if season_element:
        return season_element.text.strip().split()[0]
    return ""

def get_school_name(url):
    url_parts = url.split("/")
    return url_parts[-3]


def get_table_data(table):
    header_row = [cell.text.strip() for cell in table.select("thead tr th")]
    data_rows = table.select("tbody tr")
    data = [[cell.text.strip() for cell in row.select("th, td")] for row in data_rows]
    return pd.DataFrame(data, columns=header_row)


for yrs in range(2000,2001):
    ratings_url = f"https://www.sports-reference.com/cbb/seasons/men/{yrs}-ratings.html" 
    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")
    base_url = "https://www.sports-reference.com/"

    response = requests.get(ratings_url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", id="ratings")
#     rows = table.select("tbody tr")
    # rows = table.select("tbody tr[data-row]")[:5]  # 첫 5개 행만 선택
    rows = table.select("tbody tr")[:2]  # 첫 5개 행만 선택

    # 빈 데이터프레임 생성
    combined_df = pd.DataFrame()
   # 필요한 컬럼들을 리스트로 저장
    columns_to_drop = ["#", "Class", "Pos", "Height", "Weight", "Hometown", "High School", "RSCI Top 100", "Summary"]

    for row in rows:
        link_element = row.select_one("td:nth-of-type(1) a")
        if link_element:
            link = link_element["href"]
            full_url = base_url + link

            response = requests.get(full_url)
            soup = BeautifulSoup(response.content, "html.parser")

            roster_table = soup.find("table", id="roster")
            per_game_table = soup.find("table", id="per_game")
            totals_table = soup.find("table", id="totals")
            per_min_table = soup.find("table", id="per_min")
            per_poss_table = soup.find("table", id="per_poss")
            advanced_table = soup.find("table", id="advanced")

            data_frames = list()

            if roster_table is not None:
                roster_data = get_table_data(roster_table)
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                roster_data["Season"] = season
                roster_data["School"] = school
                data_frames.append(roster_data)
            else:
                print(f"{season} {school} Roster table not found.")
                pass

            if per_game_table is not None:
                per_game_data = get_table_data(per_game_table)
                per_game_data = per_game_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_game")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_game_data["Season"] = season
                per_game_data["School"] = school
                data_frames.append(per_game_data)
            else:
                print(f"{season} {school} PerGame table not found.")
                pass

            if totals_table is not None:
                totals_data = get_table_data(totals_table)
                totals_data = totals_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_totals")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                totals_data["Season"] = season
                totals_data["School"] = school
                # print("Totals data frame:")
                # print(totals_data.head())
                data_frames.append(totals_data)
            else:
                print(f"{season} {school} Totals table not found.")
                pass

            if per_min_table is not None:
                per_min_data = get_table_data(per_min_table)
                per_min_data = per_min_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_min")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_min_data["Season"] = season
                per_min_data["School"] = school
                # print("Per 40 Min data frame:")
                # print(per_min_data.head())
                data_frames.append(per_min_data)
            else:
                print(f"{season} {school} Per40Min table not found.")
                pass

            if per_poss_table is not None:
                per_poss_data = get_table_data(per_poss_table)
                per_poss_data = per_poss_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_per_poss")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                per_poss_data["Season"] = season
                per_poss_data["School"] = school
                # print("Per 100 Poss data frame:")
                # print(per_poss_data.head())
                data_frames.append(per_poss_data)
            else:
                print(f"{season} {school} Per100Poss table not found.")
                pass

            if advanced_table is not None:
                advanced_data = get_table_data(advanced_table)
                advanced_data = advanced_data.rename(columns=lambda x: x if x in ["Player", "Season", "School"] else x + "_advanced")
                season = get_season_from_page(soup)
                school = get_school_name(full_url)
                advanced_data["Season"] = season 
                advanced_data["School"] = school
                # print("Advanced data frame:")
                # print(advanced_data.head())
                data_frames.append(advanced_data)
            else:
                print(f"{season} {school} Advanced table not found.")
                pass

            combined_df = roster_data.copy()
            for col in columns_to_drop:
                try:
                    combined_df = combined_df.drop(col, axis=1)
                except:
                    continue

            for df in data_frames:
                if df is not None:
                    df = df.loc[:, ~df.columns.duplicated()]
                    combined_df = pd.merge(combined_df, df, on=["Player", "Season", "School"], how="outer")

            combined_df = combined_df[["Season", "School"] + [col for col in combined_df.columns if col not in ["Season", "School"]]]
            # Season 값 형식 변경
            # combined_df["Season"] = combined_df["Season"].apply(lambda x: f"{x[:4]}-{x[-2:]}")
            combined_df["Season"] = combined_df["Season"].astype(str)

            # 파일명 생성
            filename = "NCAA_" + season + ".csv"
            #  # 빈 데이터프레임을 CSV 파일로 저장
            # df.to_csv(filename, index=False)

            # 파일이 이미 존재하는 경우
            if filename in os.listdir():
                # 기존 파일 읽어오기
                existing_df = pd.read_csv(filename)
                # 기존 데이터와 현재 데이터 병합
                combined_df = pd.concat([existing_df, combined_df], ignore_index=False)
            # else:
                # combined_df = pd.concat([combined_df] + data_frames, ignore_index=False)

            # CSV 파일로 저장
            combined_df.to_csv(filename, index=False)
            print("Combined DataFrame has been saved as", filename)

            # 일정 시간 대기
            time.sleep(5)  # 5초 대기

print("Crawling complete!")
    

질문3.

1. 해당 웹페이지는 구글계정 로그인이 필요한 페이지입니다.(유료사이트)

2. 목적: 첫번째 접속 페이지 내의 테이블안에 있는 player에 링크로 연결된 url로 이동 후 데이터 수집

3. 문제상황: 링크로 연결된 url로 가서 데이터를 가져 오고 있지만 다시 처음의 페이지로 돌아와 다음 행의 선수 링크 url로 이동하지 못하고 있음. (에러 첨부)


In [ ]:
#수정 전 코드
#draft list의 선수별 데이터 가져오기(career length)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
import undetected_chromedriver as uc
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# # sheet 생성
# wb = openpyxl.Workbook()
# sheet = wb.active
# sheet.append(["no", "displayName", "team", "officialPosition", "averageSalary"])


def init_driver():
    browser = uc.Chrome()
    browser.get('https://velog.io')
    return browser

# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(browser):
    browser.find_element(By.XPATH,'//button[text()="로그인"]').click()
  
    google_login = browser.find_element(By.XPATH, '//*[@id="root"]/div[4]/div/div[2]/div[2]/div/div[1]/section[2]/div/a[2]').get_attribute('href')         
    browser.get(google_login)
    email_input = browser.find_element(By.XPATH, '//*[@id="identifierId"]')
    email_input.send_keys('구글계정')
    browser.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button').click()
    time.sleep(7)
    password_input = browser.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
    password_input.send_keys('구글계정비번')     
    browser.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button').click()
    time.sleep(15)
    
if  __name__  ==  "__main__" :
    browser = init_driver()
    do_login(browser)
    url = 'https://stathead.com/basketball/draft_finder.cgi?request=1&order_by_asc=1&is_active=N&order_by=year_id&year_max=2020&year_min=2000&offset=0'

    while True:
        # Load the page
        browser.get(url)
        target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
        target_element_login.click()
        target_element_login2 = browser.find_element(By.XPATH, '//*[@id="sh-login-google"]')
        target_element_login2.click()
        browser.get(url)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'stats')))  # Wait for the table to load

        # Extract the data from the table
        table = browser.find_element(By.XPATH, '//*[@id="stats"]')
        trs = table.find_elements(By.XPATH, './/tr')
        
#         rows = table.find_elements(By.XPATH, './/tr')[0:]  # Exclude header row   
        
        data_list = []


        for tr in trs:        
            try:
                url_cell = tr.find_element(By.XPATH, '//*[@id="stats"]/tbody/tr[1]/td[5]')
                player_url = url_cell.find_element(By.TAG_NAME, 'a').get_attribute('href')
#             except StaleElementReferenceException:
                
                browser.get(player_url)
                try:
                    more_bio_button = browser.find_element(By.XPATH, '//*[@id="meta_more_button"]')
                    more_bio_button.click()
                except NoSuchElementException:
                    pass  # Continue without clicking if "More Bio" button is not found             
             # Extract the data from the player page
                data_element = browser.find_element(By.CSS_SELECTOR, '#meta div:nth-of-type(2)')
                data = data_element.text.strip()
                data_list.append(data)
                print(data_list)
                time.sleep(5)
############################################################################################################################################################################### 
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################  
#문제 부분                 
                # Go back to the previous page
                browser.back()   #//*[@id="stats"]/tbody/tr[2]/td[5]
                time.sleep(5)
                continue               
                
            except NoSuchElementException:
                continue
        
        
        df = pd.DataFrame(data_list)
        
        # Print the DataFrame
        print(df)

        # Check if there is a next page
        next_button = browser.find_element(By.CSS_SELECTOR, '#stathead_results div:last-of-type a')
        if not next_button:
            break

        # Get the URL of the next page
        next_url = next_button.get_attribute('href')

        # Build the full URL for the next page
        url = f'https://stathead.com{next_url}'

    # Step 4: Process is complete
    print("Crawling complete!")

    # Quit the WebDriver
    driver.quit()
    